In [1]:
import tensorflow as tf
import torch
import numpy as np
import cv2
import threading


In [2]:
# Corrected Singleton Metaclass
class Singleton(type):
    _instances = {}
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            cls._instances[cls] = super().__call__(*args, **kwargs)  # Corrected super() call
        return cls._instances[cls]

# Model Loader Factory (same as before)
class ModelLoaderFactory:
    loaders = {

               "keras" : lambda path: tf.keras.models.load_model(path),
               "h5": lambda path: tf.keras.models.load_model(path),
               "pb": lambda path: tf.saved_model.load(path),
               "pt": lambda path: torch.jit.load(path)}

    @staticmethod
    def get_loader(extension):
        loader = ModelLoaderFactory.loaders.get(extension.lower())
        if loader is None:
            raise ValueError(f"Unsupported model format: {extension}")
        return loader



In [3]:
# Preprocessing Strategies (Singleton) (same as before)
class PreprocessingStrategy(metaclass=Singleton):
    def apply(self, image):
        raise NotImplementedError("Subclasses must implement apply method.")

class CataractPreprocessing(PreprocessingStrategy):
    def apply(self, image):
        image = cv2.resize(image, (224, 224))
        return cv2.convertScaleAbs(image, alpha=1.0, beta=50)

class DiabetesPreprocessing(PreprocessingStrategy):
    def apply(self, image):
        image = cv2.resize(image, (224, 224))
        green_channel = image[:, :, 1]
        red_free_image = cv2.merge([green_channel, green_channel, green_channel])
        return cv2.convertScaleAbs(red_free_image, alpha=1.5, beta=50)

class GlaucomaPreprocessing(PreprocessingStrategy):
    def apply(self, image):
        image = cv2.resize(image, (224, 224))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = image / 255.0
        return image.astype(np.float32)

class HypertensionPreprocessing(PreprocessingStrategy):
    def apply(self, image):
        image = cv2.resize(image, (224, 224))
        green_channel = image[:, :, 1]
        red_free_image = cv2.equalizeHist(green_channel)
        edges = cv2.Canny(red_free_image, 50, 150)
        blurred = cv2.GaussianBlur(red_free_image, (5, 5), 0)
        return np.stack([red_free_image, edges, blurred], axis=-1)

class PathologicalMyopiaPreprocessing(PreprocessingStrategy):
    def apply(self, image):
        return cv2.resize(image, (224, 224))

class AgeIssuesPreprocessing(PreprocessingStrategy):
    def apply(self, image):
        image = cv2.resize(image, (224, 224))
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(gray_image)
        image_faf = cv2.cvtColor(enhanced_image, cv2.COLOR_GRAY2BGR)
        edges = cv2.Canny(image, 100, 200)
        return cv2.addWeighted(image, 0.8, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 0.2, 0)


In [4]:
# Eyes Model (same as before)
class EyesModel:
    _model_cache = {}
    _cache_lock = threading.Lock()


    def __init__(self, model_path, strategy):
        self.model_path = model_path
        self.strategy = strategy
        self.model = self._load_model()


    def __eq__(self, other):
        # Two models are considered equal if they share the same path and strategy
        return isinstance(other, EyesModel) and \
               self.model_path == other.model_path and \
               type(self.strategy) == type(other.strategy)

    def __hash__(self):
        # Hash based on model path and strategy type
        return hash((self.model_path, type(self.strategy)))


    def __str__(self):
        return f"EyesModel(model_path={self.model_path}, strategy={type(self.strategy)})"

    def _load_model(self):
        with EyesModel._cache_lock:
            if self.model_path not in EyesModel._model_cache:
                extension = self.model_path.split('.')[-1]
                loader = ModelLoaderFactory.get_loader(extension)
                if loader:
                    EyesModel._model_cache[self.model_path] = loader(self.model_path)
            return EyesModel._model_cache[self.model_path]

    def diagnose(self, left_image, right_image):
        left_processed = self.strategy.apply(left_image)
        right_processed = self.strategy.apply(right_image)
        left_result = self.model.predict(np.expand_dims(left_processed, axis=0))[0]
        right_result = self.model.predict(np.expand_dims(right_processed, axis=0))[0]
        return left_result, right_result


In [ ]:
"""
    def diagnose(self, left_image, right_image):
        left_processed = self.strategy.apply(left_image)
        right_processed = self.strategy.apply(right_image)
        left_result = self.model.predict(np.expand_dims(left_processed, axis=0))[0]
        right_result = self.model.predict(np.expand_dims(right_processed, axis=0))[0]
        return left_result, right_result
for eyes_model
""""

""""
# Diagnoser (Singleton) (same as before)
class Diagnoser(metaclass=Singleton):
    def __init__(self):
        #self.models = []
        self.models = set()  # Use a set to store models and ensure uniqueness

    def add_model(self, model):
        # Models will be automatically deduplicated based on __hash__ and __eq__
        self.models.add(model)
        #self.models.append(model)

    def predict(self, left_image, right_image):
        # Serial execution to ensure order is maintained
        results = [model.diagnose(left_image, right_image) for model in self.models]
        return results

        """

In [29]:
from concurrent.futures import ThreadPoolExecutor

class Diagnoser:
    def __init__(self):
        # Use a list to store models and ensure uniqueness
        self.models = []

    def add_model(self, model):
        """
        Add a model to the Diagnoser, ensuring uniqueness based on __hash__ and __eq__.
        """
        if model not in self.models:
            self.models.append(model)

    def predict(self, left_image, right_image, parallel=False):
        """
        Generate predictions using all models.

        Args:
        - left_image: Image for the left eye.
        - right_image: Image for the right eye.
        - parallel (bool): Whether to execute predictions in parallel.

        Returns:
        - results (list): Predictions from all models in the correct order.
        """
        if parallel:
            # Map each model to its index to maintain order
            model_to_index = {model: idx for idx, model in enumerate(self.models)}

            # Function to process a model and preserve index
            def process_model(model):
                return model_to_index[model], model.diagnose(left_image, right_image)

            # Initialize a list to store results in the correct order
            results = [None] * len(self.models)

            # Execute predictions in parallel with ThreadPoolExecutor
            with ThreadPoolExecutor() as executor:
                for index, result in executor.map(process_model, self.models):
                    results[index] = result  # Store results at respective indices

        else:
            # Serial execution ensures order naturally
            results = [model.diagnose(left_image, right_image) for model in self.models]

        return results


In [ ]:

      """
    def predict(self, left_image, right_image):
        #from concurrent.futures import ThreadPoolExecutor
        #with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda model: model.diagnose(left_image, right_image), self.models))
        return results
      """


In [ ]:
"""
# Example Usage
if __name__ == "__main__":
    def load_example_image(path):
        return np.random.rand(224, 224, 3).astype(np.uint8)

    left_image_data = load_example_image("left_eye.jpg")
    right_image_data = load_example_image("right_eye.jpg")

    diagnoser = Diagnoser()
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/cataract_model_wiht_filter.keras", CataractPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/diabetic_model_wiht_filter.keras", DiabetesPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/glaucoma_model_wiht_filter.keras", GlaucomaPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/hyper_model_wiht_filter.keras", HypertensionPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/Myopia_model_2.h5", PathologicalMyopiaPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/age_model_wiht_filter.keras", AgeIssuesPreprocessing()))

    predictions = diagnoser.predict(left_image_data, right_image_data)
    print("Predictions:", predictions)

"""

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split

# Assuming Diagnoser, EyesModel, and Preprocessing strategies are defined as in the previous response

# Step 1: Loading the dataset and organizing image paths
os.environ["KAGGLE_USERNAME"] = "magdhndi"
os.environ["KAGGLE_KEY"] = "9b6c8953fb75d807a407f863ae22edc6"

!kaggle datasets download andrewmvd/ocular-disease-recognition-odir5k
!unzip ocular-disease-recognition-odir5k


In [7]:
df = pd.read_csv("/content/full_df.csv")

cach_df = df
df.head()

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,filepath,labels,target,filename
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",0_right.jpg
1,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['N'],"[1, 0, 0, 0, 0, 0, 0, 0]",1_right.jpg
2,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",2_right.jpg
3,4,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",4_right.jpg
4,5,50,Female,5_left.jpg,5_right.jpg,moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,0,../input/ocular-disease-recognition-odir5k/ODI...,['D'],"[0, 1, 0, 0, 0, 0, 0, 0]",5_right.jpg


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
df = df.head(1)
dataset_dir = "/content/ODIR-5K/ODIR-5K/Training Images"
image_size = 224

diagnoser = Diagnoser()
diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/cataract_model_wiht_filter.keras", CataractPreprocessing()))
diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/diabetic_model_wiht_filter.keras", DiabetesPreprocessing()))
diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/glaucoma_model_wiht_filter.keras", GlaucomaPreprocessing()))
diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/hyper_model_wiht_filter.keras", HypertensionPreprocessing()))
diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/Myopia_model_2.h5", PathologicalMyopiaPreprocessing()))
diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/age_model_wiht_filter.keras", AgeIssuesPreprocessing()))


In [31]:
for model in diagnoser.models:
    print(type(model.strategy))  # Prints the class of each strategy

<class '__main__.CataractPreprocessing'>
<class '__main__.DiabetesPreprocessing'>
<class '__main__.GlaucomaPreprocessing'>
<class '__main__.HypertensionPreprocessing'>
<class '__main__.PathologicalMyopiaPreprocessing'>
<class '__main__.AgeIssuesPreprocessing'>


In [37]:
def load_and_predict(left_image_path, right_image_path):
    """Loads, preprocesses, and predicts using Diagnoser for a pair of images."""
    try:
        left_image = cv2.imread(left_image_path, cv2.IMREAD_COLOR)
        right_image = cv2.imread(right_image_path, cv2.IMREAD_COLOR)

        if left_image is None or right_image is None:
            print(f"Warning: Could not read one or both images.")
            return None

        left_image = cv2.resize(left_image, (image_size, image_size))
        right_image = cv2.resize(right_image, (image_size, image_size))

        prediction = diagnoser.predict(left_image, right_image, parallel=True)  # Parallel execution with guaranteed order
        return prediction

    except Exception as e:
        print(f"Error processing images: {e}")
        return None


In [38]:
import pandas as pd
from tqdm import tqdm
import os

def process_in_batches(df, start_row, batch_size, dataset_dir, output_file="predictions.csv"):
    """
    Processes rows in the DataFrame in batches and saves predictions to a CSV file.

    Args:
    - df (pd.DataFrame): Input DataFrame with records to process.
    - start_row (int): Starting row index.
    - batch_size (int): Number of rows to process in each batch.
    - dataset_dir (str): Directory containing the images.
    - output_file (str): Path to the output CSV file.

    Returns:
    - None
    """
    # Ensure start_row and batch_size are valid
    assert start_row >= 0, "Start row must be non-negative"
    assert batch_size > 0, "Batch size must be a positive integer"

    # Slice the DataFrame for the given batch
    end_row = min(start_row + batch_size, len(df))
    batch_df = df.iloc[start_row:end_row]

    # Initialize results list
    results = []

    # Process each row in the batch
    for index, row in tqdm(batch_df.iterrows(), total=batch_df.shape[0], desc=f"Processing rows {start_row} to {end_row}"):
        age = row['Patient Age']
        sex = row['Patient Sex']
        left_fundus = os.path.join(dataset_dir, row['Left-Fundus'])
        right_fundus = os.path.join(dataset_dir, row['Right-Fundus'])
        correct_diagnosis = [row['N'], row['D'], row['G'], row['C'], row['A'], row['H'], row['M']]

        # Perform prediction
        prediction = load_and_predict(left_fundus, right_fundus)

        if prediction:
            cataract_prob, diabetic_prob, glaucoma_prob, hyper_prob, myopia_prob, age_prob = prediction

            # Left and right eye results
            left_sorted = [cataract_prob[0][0], diabetic_prob[0][0], glaucoma_prob[0][0],
                           hyper_prob[0][0], myopia_prob[0][0], age_prob[0][0]]
            right_sorted = [cataract_prob[1][0], diabetic_prob[1][0], glaucoma_prob[1][0],
                            hyper_prob[1][0], myopia_prob[1][0], age_prob[1][0]]

            # Append results
            results.append([age, sex] + left_sorted + right_sorted + correct_diagnosis)



    return results

In [39]:
# Example Usage
# Assuming `df` is already defined and loaded
results = process_in_batches(cach_df, start_row=0, batch_size=5, dataset_dir="/content/ODIR-5K/ODIR-5K/Training Images", output_file="predictions.csv")


Processing rows 0 to 5:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 899ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


Processing rows 0 to 5:  20%|██        | 1/5 [00:07<00:29,  7.27s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 962ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Processing rows 0 to 5:  40%|████      | 2/5 [00:13<00:20,  6.82s/it]

...

Processing rows 0 to 5:  60%|██████    | 3/5 [00:22<00:15,  7.59s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 999ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Processing rows 0 to 5:  80%|████████  | 4/5 [00:28<00:07,  7.09s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


Processing rows 0 to 5: 100%|██████████| 5/5 [00:38<00:00,  7.66s/it]


In [40]:
output_file="predictions.csv"
start_row=0
end_row = 5
# Assertions for the batch results
#assert len(results) == len(batch_df), f"Results length mismatch: expected {len(batch_df)}, got {len(results)}"

# Define column names
columns = ['Age', 'Sex', 'Left_cataract', 'Left_diabetic', 'Left_glaucoma', 'Left_hyper', 'Left_Myopia', 'Left_age',
            'Right_cataract', 'Right_diabetic', 'Right_glaucoma', 'Right_hyper', 'Right_Myopia', 'Right_age',
            'N', 'D', 'G', 'C', 'A', 'H', 'M']

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=columns)

# Display a preview of the results
print(results_df.head())

# Save results to a CSV file
if os.path.exists(output_file):
    # Append without overwriting the file
    results_df.to_csv(output_file, mode='a', header=False, index=False)
else:
    # Create a new file
    results_df.to_csv(output_file, index=False)

print(f"Batch from {start_row} to {end_row} processed and saved to {output_file}")


   Age     Sex  Left_cataract  Left_diabetic  Left_glaucoma    Left_hyper  \
0   69  Female   1.000000e+00   2.546632e-06       0.175294  6.784076e-10   
1   57    Male   3.026999e-09   1.389067e-01       0.221712  1.051806e-07   
2   42    Male   9.973556e-01   9.979500e-01       0.198193  6.392049e-11   
3   53    Male   8.908896e-01   2.037923e-07       0.343581  2.397901e-11   
4   50  Female   6.744714e-17   2.790738e-05       0.192183  2.162522e-10   

    Left_Myopia      Left_age  Right_cataract  Right_diabetic  ...  \
0  4.678401e-17  3.261969e-02    6.574120e-02        0.000059  ...   
1  3.471462e-22  3.751020e-03    4.485955e-12        0.003587  ...   
2  1.102241e-08  9.995713e-01    1.676708e-02        0.559096  ...   
3  1.864905e-09  9.999897e-01    9.997150e-01        0.026577  ...   
4  1.186832e-30  1.511544e-07    6.442329e-19        0.956502  ...   

    Right_hyper  Right_Myopia     Right_age  N  D  G  C  A  H  M  
0  2.096240e-10  5.021408e-19  5.854326e-08  0  0

In [89]:
from tqdm import tqdm
import numpy as np
import os

results = []

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    age = row['Patient Age']
    sex = row['Patient Sex']
    left_fundus = os.path.join(dataset_dir, row['Left-Fundus'])
    right_fundus = os.path.join(dataset_dir, row['Right-Fundus'])
    correct_diagnosis = [row['N'], row['D'], row['G'], row['C'], row['A'], row['H'], row['M']]

    # Perform prediction
    prediction = load_and_predict(left_fundus, right_fundus)

    if prediction:
        cataract_prob, diabetic_prob, glaucoma_prob, hyper_prob, myopia_prob, age_prob = prediction

        # Assertions for prediction format
        assert isinstance(prediction, (list, tuple)), f"Prediction should be a list/tuple, got {type(prediction)}"
        assert len(prediction) == 6, f"Prediction should have 6 elements, got {len(prediction)}"

        # Assertions for individual probabilities
        for i, prob in enumerate(prediction):
            assert len(prob) == 2, f"Each prediction entry should have left and right results, element {i} has {len(prob)}"
            assert all(isinstance(val, np.float32) for val in prob[0]), f"Left results should be floats, got {prob[0]}"
            assert all(isinstance(val, np.float32) for val in prob[1]), f"Right results should be floats, got {prob[1]}"

        # Left and Right eye results
        left_sorted = [cataract_prob[0][0], diabetic_prob[0][0], glaucoma_prob[0][0],
                       hyper_prob[0][0], myopia_prob[0][0], age_prob[0][0]]
        right_sorted = [cataract_prob[1][0], diabetic_prob[1][0], glaucoma_prob[1][0],
                        hyper_prob[1][0], myopia_prob[1][0], age_prob[1][0]]

        # Assertions for left and right sorted lists
        assert len(left_sorted) == 6, f"Left results should contain 6 elements, got {len(left_sorted)}"
        assert len(right_sorted) == 6, f"Right results should contain 6 elements, got {len(right_sorted)}"
        assert all(isinstance(val, np.float32) for val in left_sorted), "Left results should only contain float values"
        assert all(isinstance(val, np.float32) for val in right_sorted), "Right results should only contain float values"

        # Append results
        results.append([age, sex] + left_sorted + right_sorted + correct_diagnosis)

        # Assertions for final results structure
        assert len(results[-1]) == 2 + 6 + 6 + len(correct_diagnosis), (
            f"Each result entry should have {2 + 6 + 6 + len(correct_diagnosis)} elements, "
            f"got {len(results[-1])}"
        )


Processing rows:   0%|          | 0/1 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 633ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 620ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 798ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 775ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step


Processing rows: 100%|██████████| 1/1 [00:03<00:00,  3.89s/it]


In [79]:
cach_df.shape (6392, 19)



(6392, 19)

In [14]:
# Assertions for the overall results list
assert len(results) == df.shape[0], f"Results should have {df.shape[0]} entries, got {len(results)}"
columns = ['Age', 'Sex', 'Left_cataract', 'Left_diabetic', 'Left_glaucoma', 'Left_hyper', 'Left_Myopia', 'Left_age',
           'Right_cataract', 'Right_diabetic', 'Right_glaucoma', 'Right_hyper', 'Right_Myopia', 'Right_age',
           'N', 'D', 'G', 'C', 'A', 'H', 'M']
results_df = pd.DataFrame(results, columns=columns)



# Save to CSV
#results_df.to_csv("predictions.csv", index=False)


NameError: name 'results' is not defined

In [13]:
# Display DataFrame
results_df.head()

NameError: name 'results_df' is not defined

In [77]:
# Build DataFrame
columns = ['Age', 'Sex', 'Left_cataract', 'Left_diabetic', 'Left_glaucoma', 'Left_hyper', 'Left_Myopia', 'Left_age',
           'Right_cataract', 'Right_diabetic', 'Right_glaucoma', 'Right_hyper', 'Right_Myopia', 'Right_age',
           'N', 'D', 'G', 'C', 'A', 'H', 'M']
results_df = pd.DataFrame(results, columns=columns)

# Display DataFrame
results_df.head()

# Save to CSV
results_df.to_csv("predictions.csv", index=False)

In [ ]:
results_df.head()

In [ ]:
"""
# Step 2: Reading images, applying preprocessing, and using Diagnoser
dataset_dir = "/content/ODIR-5K/ODIR-5K/Training Images"
image_size = 224

def load_and_predict(image_list):
    #Loads, preprocesses, and predicts using Diagnoser for a list of images
    diagnoser = Diagnoser()
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/cataract_model_wiht_filter.keras", CataractPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/diabetic_model_wiht_filter.keras", DiabetesPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/glaucoma_model_wiht_filter.keras", GlaucomaPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/hyper_model_wiht_filter.keras", HypertensionPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/Myopia_model_2.h5", PathologicalMyopiaPreprocessing()))
    diagnoser.add_model(EyesModel("/content/drive/MyDrive/F5_project/age_model_wiht_filter.keras", AgeIssuesPreprocessing()))

    predictions = []
    for img_file in tqdm(image_list, desc="Predicting images"):
        image_path = os.path.join(dataset_dir, img_file)
        try:
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            if image is None:
                print(f"Warning: Could not read image at {image_path}")
                predictions.append(None) # or some default value
                continue
            image = cv2.resize(image, (image_size, image_size))
            prediction = diagnoser.predict(image, image) # Assuming both eyes are
            prediction = diagnoser.predict(image, image) # Assuming both eyes are the same for simplicity
            predictions.append(prediction)
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            predictions.append(None) # or some default value
    return predictions

    """

Mounted at /content/drive


In [ ]:
# Example: Predicting Myopia images
myopia_predictions = load_and_predict(myopia[:3])

# Example: Display first prediction
if myopia_predictions and myopia_predictions[0]:
    print("Example Myopia Prediction:", myopia_predictions[0])
else:
    print("No predictions were made.")



8.5612805e-14

In [93]:
cat_model = EyesModel("/content/drive/MyDrive/F5_project/cataract_model_wiht_filter.keras", CataractPreprocessing())

In [102]:
glu_model = EyesModel("/content/drive/MyDrive/F5_project/glaucoma_model_wiht_filter.keras", GlaucomaPreprocessing())

In [94]:
age, sex, left_fundus, right_fundus, correct_diagnosis = 0,0, 0,0, 0
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing rows"):
    age = row['Patient Age']
    sex = row['Patient Sex']
    left_fundus = os.path.join(dataset_dir, row['Left-Fundus'])
    right_fundus = os.path.join(dataset_dir, row['Right-Fundus'])
    correct_diagnosis = [row['N'], row['D'], row['G'], row['C'], row['A'], row['H'], row['M']]

Processing rows: 100%|██████████| 1/1 [00:00<00:00, 258.80it/s]


In [97]:
left_image = cv2.imread(left_fundus, cv2.IMREAD_COLOR)
right_image = cv2.imread(right_fundus, cv2.IMREAD_COLOR)

left_image = cv2.resize(left_image, (image_size, image_size))
right_image = cv2.resize(right_image, (image_size, image_size))


In [98]:
cat_res = cat_model.diagnose(left_image, right_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step


In [101]:
cat_res[0][0]

1.0

In [103]:
glu_res = glu_model.diagnose(left_image, right_image)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 272ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step


In [104]:
glu_res[0][0]

0.17529409

In [ ]:


# Example: Predicting Cataract images
cataract_predictions = load_and_predict(cataract)

# Example: Display first prediction
if cataract_predictions and cataract_predictions[0]:
    print("Example Cataract Prediction:", cataract_predictions[0])
else:
    print("No predictions were made.")

# Example: Predicting Diabetes images
diabetes_predictions = load_and_predict(diab)

# Example: Display first prediction
if diabetes_predictions and diabetes_predictions[0]:
    print("Example Diabetes Prediction:", diabetes_predictions[0])
else:
    print("No predictions were made.")

# Example: Predicting Glaucoma images
glaucoma_predictions = load_and_predict(glaucoma)

# Example: Display first prediction
if glaucoma_predictions and glaucoma_predictions[0]:
    print("Example Glaucoma Prediction:", glaucoma_predictions[0])
else:
    print("No predictions were made.")

# Example: Predicting Hypertension images
hypertension_predictions = load_and_predict(hyper)

# Example: Display first prediction
if hypertension_predictions and hypertension_predictions[0]:
    print("Example Hypertension Prediction:", hypertension_predictions[0])
else:
    print("No predictions were made.")

# Example: Predicting Age issues images
age_predictions = load_and_predict(age)

# Example: Display first prediction
if age_predictions and age_predictions[0]:
    print("Example Age issues Prediction:", age_predictions[0])
else:
    print("No predictions were made.")

# Example: Predicting other images
other_predictions = load_and_predict(other)

# Example: Display first prediction
if other_predictions and other_predictions[0]:
    print("Example Other issues Prediction:", other_predictions[0])
else:
    print("No predictions were made.")

# Example: Predicting Normal images
normal_predictions = load_and_predict(normal)

# Example: Display first prediction
if normal_predictions and normal_predictions[0]:
    print("Example Normal issues Prediction:", normal_predictions[0])
else:
    print("No predictions were made.")